In [2]:
import pandas as pd
import time
import sys
import gc
import os

from dotenv import load_dotenv

### Generación de datos:

#### Definición:

Este proyecto está dirigido a la creación de agentes útiles en el área de investigación de IOCs en ciberseguridad. 

Un agente se entiende en este sentido, como la combinación de un LLM, un prompt y piezas de código que puede ejecutar según el caso lo requiera, conocidas como herramientas. Actualmente se han definido diferentes arquitecturas, diseñadas para permitir la interacción entre agentes para cumplir una tarea bien definida, así como otras más simples que buscan aumentar las capacidades del LLM para obtener la información que le permita llevar a cabo el requerimiento del prompt inicial. 



In [ ]:
class DataGeneration(dtaq.IOCValidatedModel):

    def generation_API_call(self):
        """Function to evaluate the data of APIs using a document to extract information to analyze the
        behaviour of information that is brought by each CTI web tool"""

        ioc_value, ioc_type = self.ioc
        if ioc_type == "ip":
            ## Virus Total:
            virustotal = dtaq.VirusTotal(ioc=ioc_value).display_ip_info()

            ## AbuseIPDB
            abuseipdb = dtaq.AbuseIPDB(ioc=ioc_value).check_ip()
            
            ## Whois
            whois_adap = dtaq.WHOIS_RDAP(ioc=ioc_value).get_whois_data()

            ## Shodan IP
            shodan = dtaq.ShodanIO(ioc=ioc_value).search_data_in_shodan()

            return self.dataframe_report(ioc_value, virustotal, abuseipdb, whois_adap, shodan)


        elif ioc_type == "domain":

            ## Virus Total:
            virustotal = dtaq.VirusTotal(ioc=ioc_value).display_domain_info()
            ## AbuseIPDB
            abuseipdb = "No aplica"
            ## Whois
            whois_adap = "No aplica"
            ## Shodan Domain
            shodan = dtaq.ShodanIO(ioc=ioc_value).search_data_in_shodan()

            return self.dataframe_report(ioc_value, virustotal, abuseipdb, whois_adap, shodan)
        
        else:
            ## Virus Total:
            virustotal = dtaq.VirusTotal(ioc=ioc_value)

            ## AbuseIPDB
            abuseipdb = dtaq.AbuseIPDB(ioc=ioc_value).check_ip()
            
            ## Whois
            whois_adap = dtaq.WHOIS_RDAP(ioc=ioc_value).get_whois_data()

            ## Shodan IP
            shodan = dtaq.ShodanIO(ioc=ioc_value).search_data_in_shodan()
            ## Shodan Domain
            if shodan == {}:
                shodan = dtaq.ShodanIO(ioc=ioc_value).search_data_in_shodan()

            return self.dataframe_report(ioc_value, virustotal, abuseipdb, whois_adap, shodan)

            

    def dataframe_report(self, ioc_value, virustotal, abuseipdb, whois_adap, shodan):
        """Function to complete the pandas data frame of responses base on iocs present in a document"""
        
        report = pd.DataFrame([{"IoC": str(ioc_value), "VirusTotal": str(virustotal), "AbuseIPDB": str(abuseipdb), "Whois_rdap": str(whois_adap), "Shodan.io": str(shodan), "Data_Time": str(self.time_report_generation())}], index=[0])

        # print(type(virustotal))
        """
        report["IoC"]   =  str(ioc_value)
        report["VirusTotal"]  = str(virustotal)
        report["AbuseIPDB"]   = str(abuseipdb)
        report["Whois_rdap"]  = str(whois_adap)
        report["Shodan.io"]   = str(shodan)
        report["Data_Time"]   = str(self.time_report_generation())
        """
        # print(report)

        return report
    
    def time_report_generation(self):
        
        s = strftime("%a_%d_%b_%Y %H-%M", localtime(time.time()))

        return s
    
    
    def createreport_toexcel(self, data):

        if not os.path.exists("scripts\\eda\\data"):
            os.makedirs("scripts\\eda\\data")

        return data.to_excel("scripts\\eda\\data\\Reporte_APIs_validation_" + str(self.time_report_generation()) + ".xlsx", sheet_name="Reporte_APIs")


### Almacenamiento de datos: 

In [5]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017')
db = 'langchainAgent'

def query_db(collection: str, query: dict):
    collection = client.get_database(db).get_collection(collection)
    return pd.DataFrame(list(collection.find(query)))

def insert_docs_to_db(docs: dict, collection: str):
    collection = client.get_database(db).get_collection(collection)
    collection.insert_many(docs)


### Visualización de los Datos

In [6]:
df_vt = query_db("VirusTotal", {})
pd.json_normalize(df_vt.data).T.head(60)

,0,1
id,ambakgroup.com,64.137.9.118
type,domain,ip_address
links.self,https://www.virustotal.com/api/v3/domains/amba...,https://www.virustotal.com/api/v3/ip_addresses...
attributes.last_https_certificate_date,1729970117,1744386520
attributes.whois_date,1761264000,1750870838
attributes.tags,[],[]
attributes.last_update_date,1729728000.0,NaN
attributes.jarm,05d10d20d21d20d05c05d10d05d20d74fcf6501ae7a923...,21d19d00021d21d00042d43d0000005ad20eceaf7f71ae...
attributes.last_analysis_date,1752037321,1752035674
attributes.last_modification_date,1752037329,1752036333


In [7]:
df_abuse = query_db("AbuseIPDB", {})
df_abuse.T

,0
_id,686e02eebd9e2a015a58c5f9
ipAddress,64.137.9.118
isPublic,True
ipVersion,4
isWhitelisted,False
abuseConfidenceScore,0
countryCode,US
usageType,Data Center/Web Hosting/Transit
isp,Individual Entrepreneur Anton Levin
domain,vdska.online


In [8]:
df_whois = query_db("whois", {})
df_whois.T

,0
_id,686e02febd9e2a015a58c5fa
handle,64.137.9.0 - 64.137.9.255
startAddress,64.137.9.0
endAddress,64.137.9.255
ipVersion,v4
name,Voldeta
type,ASSIGNED PA
country,US
parentHandle,64.137.0.0 - 64.137.127.255
cidr0_cidrs,"[{'v4prefix': '64.137.9.0', 'length': 24}]"


In [9]:
df_shodan = query_db("Shodan", {})
df_shodan.T

,0
_id,686e0339bd9e2a015a58c5fc
matches,[]
total,0


### Exploración:

#### Propósito:

Se busca entonces, definir herramientas que pueda utilizar un LLM de forma "autónoma", para obtener más información sobre dominios o ips, asociados a indicadores de compromiso. Esto es debido a que, deendiendo de la naturaleza del IOC, la información disponible puede venir de diferentes endpoints, por ejemplo, herramientas como virustotal exponen rutas o urls diferentes para entregar información de cada tipo de indicador. 

Adicionalmente, en ciberseguridad, es común encontrar detecciones tardías, o que algunos proveedores tengan una latencia de horas, días o o incluso meses en lograr agregar contexto a los IOCs (información que sirve para entender el tipo de amenaza). Por lo que adicionalmente, se planea contar con un versionamiento de los datos, que puede habilitar otro tipo de información o comprensión de los atacantes, ya que puede servir de base para llevar a cabo investigaciones respecto al seguimiento de actores maliciosos. 

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
df_vt = pd.json_normalize(df_vt['data'])
df_vt

,id,type,links.self,attributes.last_https_certificate_date,attributes.whois_date,attributes.tags,attributes.last_update_date,attributes.jarm,attributes.last_analysis_date,attributes.last_modification_date,...,attributes.rdap.country,attributes.rdap.parent_handle,attributes.rdap.cidr0_cidrs,attributes.rdap.arin_originas0_originautnums,attributes.continent,attributes.regional_internet_registry,attributes.network,attributes.asn,attributes.country,attributes.as_owner
0,ambakgroup.com,domain,https://www.virustotal.com/api/v3/domains/amba...,1729970117,1761264000,[],1.729728e+09,05d10d20d21d20d05c05d10d05d20d74fcf6501ae7a923...,1752037321,1752037329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64.137.9.118,ip_address,https://www.virustotal.com/api/v3/ip_addresses...,1744386520,1750870838,[],NaN,21d19d00021d21d00042d43d0000005ad20eceaf7f71ae...,1752035674,1752036333,...,US,64.137.0.0 - 64.137.127.255,"[{'v4prefix': '64.137.9.0', 'length': 24, 'v6p...",[],NA,ARIN,64.137.9.0/24,49791.0,US,Newserverlife LLC


In [15]:
df_vt.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 478 columns):
 #    Column                                                                         Dtype  
---   ------                                                                         -----  
 0    id                                                                             object 
 1    type                                                                           object 
 2    links.self                                                                     object 
 3    attributes.last_https_certificate_date                                         int64  
 4    attributes.whois_date                                                          int64  
 5    attributes.tags                                                                object 
 6    attributes.last_update_date                                                    float64
 7    attributes.jarm                                        

In [16]:
df_abuse.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   _id                   1 non-null      object
 1   ipAddress             1 non-null      object
 2   isPublic              1 non-null      bool  
 3   ipVersion             1 non-null      int64 
 4   isWhitelisted         1 non-null      bool  
 5   abuseConfidenceScore  1 non-null      int64 
 6   countryCode           1 non-null      object
 7   usageType             1 non-null      object
 8   isp                   1 non-null      object
 9   domain                1 non-null      object
 10  hostnames             1 non-null      object
 11  isTor                 1 non-null      bool  
 12  totalReports          1 non-null      int64 
 13  numDistinctUsers      1 non-null      int64 
 14  lastReportedAt        1 non-null      object
dtypes: bool(3), int64(4), object(8)
memory usage

In [17]:
df_whois.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              1 non-null      object
 1   handle           1 non-null      object
 2   startAddress     1 non-null      object
 3   endAddress       1 non-null      object
 4   ipVersion        1 non-null      object
 5   name             1 non-null      object
 6   type             1 non-null      object
 7   country          1 non-null      object
 8   parentHandle     1 non-null      object
 9   cidr0_cidrs      1 non-null      object
 10  status           1 non-null      object
 11  entities         1 non-null      object
 12  links            1 non-null      object
 13  events           1 non-null      object
 14  rdapConformance  1 non-null      object
 15  notices          1 non-null      object
 16  port43           1 non-null      object
 17  objectClassName  1 non-null      object

In [18]:
df_shodan.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   _id      1 non-null      object
 1   matches  1 non-null      object
 2   total    1 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


#### Conclusión

Note que la mayoría de los campos se pueden pensar como variables categóricas (geolocalización, registrants, veredictos de diferentes proveedores de ciberseguridad, algoritmos de encripción, registros dns, entre otros) Por lo que pensar en un modelo de datos, resulta una tarea que supera el alcance de este proyecto; sin embargo, sistemas como los LLMs sobresalen al poder trabajar con información en este formato, por lo que permite extraer o imitar la base de investigación de los operadores de ciberseguridad, automatizando incluso la extracción de información, y facilitando la potencial integración con otras arquitecturas de agentes, como la de Human In the Loop (HIL), 
